In [ ]:
# Packages

import pandas as pd
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp
import pytz
from datetime import timedelta

In [ ]:

RESPECK_FILE = ''
PSG_FILE = ''
LABELS_FILE = ''

respeck_df = pd.read_csv(RESPECK_FILE)
respeck_df['timestamp'] = pd.to_datetime(respeck_df['alignedTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
respeck_df['timestamp'] = respeck_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
respeck_df.set_index('timestamp', inplace=True)

psg_df = pd.read_csv(PSG_FILE)
psg_df['timestamp'] = pd.to_datetime(psg_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
psg_df['timestamp'] = psg_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
psg_df.set_index('timestamp', inplace=True)

labels_df = pd.read_csv(LABELS_FILE)
labels_df['timestamp'] = pd.to_datetime(labels_df['UnixTimestamp'], unit='ms')
tz = pytz.timezone('Asia/Bishkek')
labels_df['timestamp'] = labels_df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(tz)
labels_df.set_index('timestamp', inplace=True)

In [ ]:

def plot_events(respeck_df, psg_df, osa_events_df, buffer_minutes=2):
    """
    Plot PSG nasal respiration and Respeck sensor data during Obstructive Apnea events
    
    Parameters:
    - respeck_df: DataFrame with Respeck sensor data (x, y, z, breathingSignal)
    - psg_df: DataFrame with PSG data (including 'Resp nasal')
    - osa_events_df: DataFrame with Obstructive Apnea events
    - buffer_minutes: Minutes to show before and after each event
    """
    
    # Check if we have any OSA events
    if osa_events_df.empty:
        print("No Obstructive Apnea events found in the data")
        return
    
    # Calculate number of events and create subplots
    n_events = len(osa_events_df)
    print(f"Found {n_events} Obstructive Apnea events")
    
    # Create subplots
    fig = sp.make_subplots(
        rows=n_events, cols=1,
        subplot_titles=[f'Obstructive Apnea Event {i+1} - {event_time.strftime("%Y-%m-%d %H:%M:%S")}' 
                       for i, (event_time, _) in enumerate(osa_events_df.iterrows())],
        specs=[[{"secondary_y": True}] for _ in range(n_events)],
        vertical_spacing=0.08
    )
    
    buffer_td = timedelta(minutes=buffer_minutes)
    
    for idx, (event_time, event_row) in enumerate(osa_events_df.iterrows()):
        row = idx + 1
        
        # Define time window around the event
        start_time = event_time - buffer_td
        end_time = event_time + buffer_td
        
        # Extract event duration if available
        duration = event_row.get('Duration', 0)
        duration = float(duration.replace(',', '.'))

        if duration > 0:
            event_end_time = event_time + timedelta(seconds=duration)
        else:
            event_end_time = event_time + timedelta(seconds=30)  # Default 30s if no duration
        
        print(f"Processing event {idx+1} at {event_time}")
        print(f"Window: {start_time} to {end_time}")
        
        # Filter data for this time window
        respeck_window = respeck_df[start_time:end_time]
        psg_window = psg_df[start_time:end_time]
        
        if respeck_window.empty and psg_window.empty:
            print(f"No data found for event {idx+1} in the specified time window")
            # Add annotation for no data
            fig.add_annotation(
                text=f'No data available for Event {idx+1}<br>{event_time}',
                xref="x domain", yref="y domain",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=12),
                row=row, col=1
            )
            continue
        
        # Plot PSG nasal respiration on primary y-axis
        if not psg_window.empty and 'Resp nasal' in psg_window.columns:
            fig.add_trace(
                go.Scatter(
                    x=psg_window.index,
                    y=psg_window['Resp nasal'],
                    mode='lines',
                    name='PSG Nasal Resp',
                    line=dict(color='blue', width=2),
                    opacity=0.8,
                    showlegend=(idx == 0)
                ),
                row=row, col=1, secondary_y=False
            )
        
        # Plot Respeck data on secondary y-axis
        if not respeck_window.empty:
            # Plot x, y, z accelerometer data
            if 'x' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['x'],
                        mode='lines',
                        name='Respeck X',
                        line=dict(color='red', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            if 'y' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['y'],
                        mode='lines',
                        name='Respeck Y',
                        line=dict(color='green', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            if 'z' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['z'],
                        mode='lines',
                        name='Respeck Z',
                        line=dict(color='magenta', width=1),
                        opacity=0.7,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
            
            # Plot breathing signal with different style
            if 'breathingSignal' in respeck_window.columns:
                fig.add_trace(
                    go.Scatter(
                        x=respeck_window.index,
                        y=respeck_window['breathingSignal'],
                        mode='lines',
                        name='Respeck Breathing',
                        line=dict(color='orange', width=2),
                        opacity=0.9,
                        showlegend=(idx == 0)
                    ),
                    row=row, col=1, secondary_y=True
                )
        
        # Add shaded region for apnea event
        fig.add_vrect(
            x0=event_time, x1=event_end_time,
            fillcolor="red", opacity=0.3,
            layer="below", line_width=0,
            annotation_text=f"Apnea Event ({duration:.1f}s)",
            annotation_position="top left",
            row=row, col=1
        )
        
        # Add vertical line at event start
        fig.add_vline(
            x=event_time, line_dash="dash", line_color="red",
            line_width=2, opacity=0.8,
            row=row, col=1
        )
        
        # Set x-axis range
        fig.update_xaxes(range=[start_time, end_time], row=row, col=1)
        
        # Update y-axis labels
        fig.update_yaxes(title_text="PSG Nasal Respiration", title_font_color="blue", 
                        row=row, col=1, secondary_y=False)
        fig.update_yaxes(title_text="Respeck Sensor Data", title_font_color="red", 
                        row=row, col=1, secondary_y=True)
    
    # Update layout
    fig.update_layout(
        height=600*n_events,
        title_text="Obstructive Apnea Events Analysis",
        showlegend=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    # Update x-axis labels for bottom subplot only
    fig.update_xaxes(title_text="Time", row=n_events, col=1)
    
    fig.show()
    
    # Print summary statistics
    print(f"\n=== Summary ===")
    print(f"Total Obstructive Apnea events: {n_events}")
    print(f"Time window per event: ±{buffer_minutes} minutes")
    
    if not osa_events_df.empty:
        durations = osa_events_df['Duration'].dropna()
        durations = pd.to_numeric(osa_events_df['Duration'].astype(str).str.replace(',', '.', regex=False), errors='coerce').dropna()
        if not durations.empty:
            print(f"Average event duration: {durations.mean():.1f} seconds")
            print(f"Duration range: {durations.min():.1f} - {durations.max():.1f} seconds")

In [ ]:

plot_events(respeck_df, psg_df, labels_df[labels_df['Event'] == 'Cheyne-Stokes'], buffer_minutes=4)